# Starter Sample App: Global Wine Reviews

## Overview

This sample application is a small example used to help me get acquainted with daikon processes. The notebook is exposed through an application on openshift. It loads in information about global wine reviews and then displays the mean review points for each wine per country on a coloured map. The lighter the area the higher the review score.

# Oshinko

This first cell connects to a local spark session and enter the config for the postgresql driver.

## Review

### Advantages:

Creating the oshinko template and web-ui is very easy and simple to do.

The webui enables fast and easy deployment of a spark cluster.

Tutorials easy to get up and running and help to explain the technologies.

### Disadvantages 

I kept running into an intermittent connection problem with spark - i have not seen this again since i cleaned my docker images.

Oshinko cluster doesn't find postgresql driver there must be some way of telling all the nodes in the spark cluster about the driver. The research on the internet was ambiguous and I have tried many of the solutions without success (any help appreciated).

The JGrafZahl and GrafZahl did not work locally for me this is either because of a networking issue on mac OS or becuase i was running 3.6.

In [1]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import mean, desc
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

sparkSession = SparkSession.builder.master("local[*]") \
.config("spark.driver.extraClassPath","/opt/postgresql-42.1.4.jar") \
.getOrCreate()

# Importing Data

This cell imports data from a local csv file, which contains information about different wines across the world the two main data points this application focuses on is the country and the number of points each wine received for that country. The wine review data is then written into a Postgresql database hosted on openshift. 

## Review

### Advantages

The postgresql docker image is very easy to pull and deploy.

The open shift web page containing these instructions is easy to follow.

### Disadvantages

The only problem with importing data is when you upload it to a cluster you need to state an endpoint, which you may not have if you are simply trying to upload a csv file.- This is being looked at.

In [2]:
import psycopg2
conn = psycopg2.connect("host='172.17.0.4' port='5432' dbname='wineDb' user='username' password='password'")
cur = conn.cursor()
#make table
f = open(r'wineData.csv', 'r')
cur.copy_from(f, "wine_reviews", sep=',')
conn.commit()
f.close()

# Creating the Map

This wine review data is then loaded into the spark session and put into a dataframe. The country and the amount of review points the wine received is then selected and the mean taken. These are ordered and then put into lists to display in the map using plotly. 


## Review

### Advantages

It was easy to change the docker file in the base-notebook repo i forked to add in new libraries like plotly.

The data frames gave you a lot of flexibility for querying the data.

The task gave me a lot of work which helped me to review and work through the processes and technologies used by the team - and continues to do so.

## Future Work:

Make the notebook into a native cloud application.


In [3]:
import plotly.graph_objs as go

url = "jdbc:postgresql://172.17.0.4/wineDb?user=username&password=password"
df = (sparkSession.read.format("jdbc")
    .options(url=url, dbtable="wine_reviews")
    .load())

table = df.select('country','points').groupBy('country').agg(mean('points')).orderBy('avg(points)',ascending=False)
countryCols = table.select('country').collect()
countries = list()
for country in countryCols:
    countries.append(str(country[0]))
pointCols = table.select('avg(points)').collect()
points = list()
for point in pointCols:
    points.append(point[0])
data =  dict(type = 'choropleth',
        locationmode='country names',
        locations = countries,
        colorscale='Blues',
        z = points,
        colorbar = {'title': 'Average Rating'}
)
layout = dict(geo = {'scope':'world'})
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)